<a href="https://colab.research.google.com/github/babelfishz/ocalendar/blob/master/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install selenium
!apt-get install chromium-browser
!apt-get install chromium-chromedriver

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import os
from pathlib import Path
import shutil

%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def init_driver():

  opts = Options()
  opts.add_argument('--no-sandbox')
  opts.add_argument('--headless')
  opts.add_argument('--disable-dev-shm-usage')
  opts.add_argument("user-agent = 'Mozilla/5.0 (Windows NT 10.0; ) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'")

  driver = webdriver.Chrome(options=opts)
  return driver

def grab(driver, name):
  url ='http://ppbc.iplant.cn'
  driver.get(url = url)

  text = driver.find_element_by_id('txt_key1')
  text.clear()
  text.send_keys(name)

  button = driver.find_element_by_id('but_search2')
  button.click()

  SCROLL_PAUSE_TIME = 3

# Get scroll height
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
      # Scroll down to bottom
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

      # Wait to load page
      time.sleep(SCROLL_PAUSE_TIME)

      # Calculate new scroll height and compare with last scroll height
      new_height = driver.execute_script("return document.body.scrollHeight")
      if new_height == last_height:
          break
      last_height = new_height

  try:
    info = driver.find_element_by_id('gdzwdesc')
    print(info.text)
  except:
    print("no descripiton")

  results = driver.find_elements_by_css_selector('div.item_t')
  urllist = []
  for result in results:
    target = result.find_element_by_tag_name('a').get_attribute('href')
    urllist.append(target)

  print(len(urllist))

  files =[]
  for url in urllist[:]:
    try:
      driver.get(url = url)
      image = driver.find_element_by_css_selector('div.divcen').find_element_by_tag_name('img')
      img_src = image.get_attribute("src")
      print(img_src)

      file = os.path.basename(img_src)
      header = {'Referer':url}
      try:
        content = requests.get(img_src, headers = header).content
        with open(file, 'wb') as f:
          f.write(content)
          files.append(file)
      except:
        print("failed to read file ", img_src)
        continue
    except:
      print("fail to read ", url)
      continue
  return files

def get_floras():
  url ="http://www.ocalendar.com.cn/api/photo/test/"
  userId = 'eyJpdiI6InNIR2ZEZUNtMjR3V292bjlBNWJoeFE9PSIsInZhbHVlIjoiUlwvUk1wcnFkUEpYNktuVXY3dnRadzNNTk0yNzhmOVcrWDNXQUtFVXY5dE1BR0MrSWVuQjRlUlQ1aXBja3VtYWwiLCJtYWMiOiJmZjYzNWY1ZWY3OTU5NTljNWQ2ZmIxNDcxOGZkYzI0N2U0ZDUyZWYwNGQwYzQ3YTlhNzVmOWUyM2I5M2MwN2Y5In0'
  
  PARAMS = {
    'userId': userId,
  } 
  response = requests.get(url=url, params = PARAMS)
  return response.json()

def read_from_file():
  index = 0
  try:
    with open('/content/gdrive/My Drive/progress/progress.txt', "r") as f:
      for line in f:
        key, value = line.strip().split()
        index = int(value)
  except:
    index = 0
  return index

def write_to_file(index):
    with open('/content/gdrive/My Drive/progress/progress.txt', "w") as f:
        string = 'index' + " " + str(index)
        f.write(string + "\n")

def copy_files(files, dst):
  Path(dst).mkdir(parents=True, exist_ok=True)
  for file in files:
    try:
      shutil.copyfile(file, dst+file)
      #print(file, dst+file) 
    except:
      print(dst + file+ "copy failed")
      continue


# plots images with labels within jupyter notebook
def plots(files, figsize=(24,12), cols=10, interp=False, titles=None):
    f = plt.figure(figsize=figsize)
    #cols = len(files)//rows if len(files) % 2 == 0 else len(files)//rows + 1
    rows = len(files)//cols + 1
    for i in range(len(files)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        img=mpimg.imread(files[i])
        plt.imshow(img)
        plt.show



In [0]:
def main():

  floras = get_floras()
  
  start = read_from_file() + 1
  driver = init_driver()
  
  for index, flora in enumerate(floras[start:]):
    name = flora['floraName']
    print(index+start+1, name)
    
    files = grab(driver,name)
    
    dst ='/content/gdrive/My Drive/ppbc/' + name + '/'
    Path(dst).mkdir(parents=True, exist_ok=True)
    copy_files(files, dst)

    write_to_file(start + index)

    #plots(files)

  driver.quit()

if __name__ == '__main__':
    main()

In [0]:
driver = init_driver()
name = '全唇盂兰'
files = grab(driver,name)
    
dst ='/content/gdrive/My Drive/ppbc/' + name + '/'
Path(dst).mkdir(parents=True, exist_ok=True)
copy_files(files, dst)



In [0]:
plots(files)